# Task1

In [84]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.cross_validation import KFold
from pprint import pprint
from time import time
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import svm
from sklearn import metrics
from sklearn.grid_search import GridSearchCV



stop = stopwords.words('english')
stemmer = PorterStemmer()


def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    if len(text) >5:
        tokenizer = RegexpTokenizer('\w+')#('\w+|\$[\d\.]+|\S+')#  #tokenizer that picks out sequences of alphanumeric characters as tokens and drops everything else
        tokens=tokenizer.tokenize(text)
        final_tokens = []
        #print(tokens)
        if len(tokens) > 2:
            for text in tokens:
                if text not in stop and len(text)>2:
                    final_tokens.append(text)
        else:
            final_tokens = tokens
        #print(final_tokens)                
        stems = stem_tokens(final_tokens, stemmer)
        #print stems
        return stems
        #return tokens
    else :
        return text.split(' ')


problem = pd.read_csv('reddit_train_top20', encoding = 'utf-8')
#print(problem[problem.title.str.len() <5])
problem = problem.drop_duplicates(take_last = True)
problem = problem.drop(problem[problem.duplicated('title') == True].index)
#problem = problem.drop(problem.title.str.len() <5)Re
problem['subreddit'] = problem['subreddit'].str.lower()
problem['title'] = problem['title'].str.lower()

In [85]:
problem.head(20)

,title,subreddit
0,[ps4] lf5m (who has) hm gate keeper cp.,fireteams
1,pov view in competitive,leagueoflegends
2,if you were given the chance to go back and re...,askreddit
3,[h] fn howl 0.04fv + mw vulcan [w] knife offers,globaloffensivetrade
4,stop pressing the button,thebutton
5,the bathmat is a towel for your feet.,showerthoughts
6,"reddit, what is your best quiz team name?",askreddit
7,two swedes experiencing norwegian svalbard,videos
8,the grass is greener on the other side of the ...,showerthoughts
9,just got a stattrak vulcan fn,globaloffensive


In [186]:

final_pipeline = Pipeline((
    ('vec', CountVectorizer(binary = True, analyzer='word',#tokenizer=tokenize,
                            lowercase=True, max_df = 0.3)),#stop_words='english',
    #('tfidf', TfidfTransformer()),
    #('vec', TfidfVectorizer(tokenizer=tokenize,
                            #max_df = 0.3,ngram_range = (1,1),norm = 'l2',use_idf = False, binary=True)),
    ('chi2', SelectKBest(chi2, k= 1000)),
    #('clf', svm.SVC()),
    ('clf', BernoulliNB()),
    #('clf', MultinomialNB()),
    #('clf', GaussianNB()),        
))
#parameters = {
    #'vec__max_df': (0.3, 0.4, 0.5),#, 0.75, 1.0),
    #'vec__ngram_range': ((1, 1), (2, 2), (2,3)),  # unigrams or bigrams
    #'vec__use_idf': (True, False),
    #'vec__norm': ('l1', 'l2'),
#}
#grid_search = GridSearchCV(final_pipeline, parameters, n_jobs=-1, verbose=1)
#t0 = time()




In [187]:
kf = KFold(problem.shape[0], n_folds=6, shuffle=True)
scores = []

for train_indices, test_indices in kf:
    train_text = problem.iloc[train_indices]['title'].values
    train_class = problem.iloc[train_indices]['subreddit'].values

    test_text = problem.iloc[test_indices]['title'].values
    test_class = problem.iloc[test_indices]['subreddit'].values

    final_pipeline.fit(train_text, train_class)
    predicted = final_pipeline.predict(test_text)
    #print(predicted)
    #print(class_test)
    print(np.mean(predicted == test_class))
    scores.append(np.mean(predicted == test_class))
    #print(metrics.classification_report(test_class, predicted))
    #grid_search.fit(train_text, train_class)
    #print("Best score: %0.3f" % grid_search.best_score_)
    #print("Best parameters set:")
    #best_parameters = grid_search.best_estimator_.get_params()
    #for param_name in sorted(parameters.keys()):
        #print("\t%s: %r" % (param_name, best_parameters[param_name]))
print 'Avg accuracy: %.2f' % (sum(scores)/len(scores))



0.601829666739
0.602918754084
0.590285340884
0.585185185185
0.593464052288
0.598474945534
Avg accuracy: 0.60


In [143]:
#Original code extracted from http://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers


def print_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        #print("%s== %s" % (class_label,
        #      " ".join(feature_names[j] for j in top10)))
        print("{:<20}: ".format(class_label)+
              " ".join(feature_names[j] for j in top10))
C_V=final_pipeline.steps[0][1] #CountVectorizer
B_NB=final_pipeline.steps[2][1] #BernoulliNB
categories=list(set(problem.subreddit.values))
print_top10(C_V,B_NB,categories)

globaloffensive     : 07pm 2050 1941 3200x1800 2015_kungfu 20mins 60s 2x1 640x480 5x
newsofiran          : 2015_kungfu 169 4600 8300 2050 3200x1800 640x480 5x 83 77
betternews          : 1vs1 1941 3200x1800 2050 281 07pm 60s 640x480 5x 2x1
videos              : 03日 1000 07pm 7k 32t 1941 5x 3200x1800 640x480 2015_kungfu
funny               : 170k 1941 83 3200x1800 07pm 2050 2015_kungfu 148 640x480 5x
thebutton           : 77 3200x1800 169 2009 640x480 07pm 2050 5x 7850k 180keys
dota2               : 1307 1955 24fv 152 800k 1559 1941 70keys 2004 40متری
adviceanimals       : 07pm 1941 83 2050 2015_kungfu 3200x1800 2x1 60s 640x480 5x
showerthoughts      : 2050 2x1 32t 07pm 1941 2015_kungfu 1972 640x480 3200x1800 5x
pics                : 157 7k 2x1 1941 2015_kungfu 07pm 32t 2050 5x 640x480
aww                 : 520 1182 10x 2x 21s 1901 195k 260x 1939 220k
askreddit           : 1972 7k 32t 07pm 1941 2050 2015_kungfu 3200x1800 640x480 5x
music               : 7k 2050 83 2015_kungfu 2v2s 07pm 

In [125]:
target_problem = pd.read_csv('reddit_test_top20', encoding = 'utf-8')
target_problem.head(20)

target_problem = target_problem.drop_duplicates(take_last = True)
target_problem = target_problem.drop(target_problem[target_problem.duplicated('title') == True].index)
target_problem['subreddit'] = target_problem['subreddit'].str.lower()
target_problem['title'] = target_problem['title'].str.lower()
predicted = final_pipeline.predict(target_problem['title'])

names=sorted(list(set(target_problem['subreddit'])))
print(np.mean(predicted == target_problem['subreddit']))
print(metrics.classification_report(target_problem['subreddit'],
                                    predicted,target_names=names))

0.603498993652
                      precision    recall  f1-score   support

       adviceanimals       0.22      0.04      0.07       821
           askreddit       0.75      0.86      0.80      5720
                 aww       0.54      0.30      0.39       959
          betternews       0.70      0.27      0.39       925
               dota2       0.82      0.39      0.53       829
   explainlikeimfive       0.89      0.91      0.90       786
           fireteams       0.99      0.99      0.99      3586
               funny       0.21      0.58      0.31      2339
              gaming       0.45      0.22      0.29      1147
     globaloffensive       0.59      0.28      0.38       949
globaloffensivetrade       0.96      0.95      0.95      2074
     leagueoflegends       0.57      0.54      0.55      2161
               music       0.76      0.43      0.55       816
                news       0.33      0.46      0.39      1650
          newsofiran       1.00      0.99      0.99   

# Use another classifiers

In [192]:
#text_train = problem['title']
#class_train = problem['subreddit']
#text_train_small, text_validation, class_train_small, class_validation = train_test_split(
#    text_train, class_train, test_size=.2, random_state=42)
pipeline2 = Pipeline((
    ('vec', CountVectorizer(#binary = True, 
                            analyzer='word',#tokenizer=tokenize,
                            lowercase=True, max_df = 0.3)),#stop_words='english',
    #('tfidf', TfidfTransformer()),
    #('vec', TfidfVectorizer(tokenizer=tokenize,
                            #max_df = 0.3,ngram_range = (1,1),norm = 'l2',use_idf = False, binary=True)),
    ('chi2', SelectKBest(chi2, k= 1000)),
    #('clf', svm.SVC()),
    #('clf', BernoulliNB()),
    ('clf', MultinomialNB(alpha=0.5)),
    #('clf', GaussianNB()),        
))





In [193]:
kf = KFold(problem.shape[0], n_folds=6, shuffle=True)
scores = []

for train_indices, test_indices in kf:
    train_text = problem.iloc[train_indices]['title'].values
    train_class = problem.iloc[train_indices]['subreddit'].values

    test_text = problem.iloc[test_indices]['title'].values
    test_class = problem.iloc[test_indices]['subreddit'].values

    pipeline2.fit(train_text, train_class)
    predicted = pipeline2.predict(test_text)
    #print(predicted)
    #print(class_test)
    print(np.mean(predicted == test_class))
    scores.append(np.mean(predicted == test_class))
    #print(metrics.classification_report(test_class, predicted))
    #grid_search.fit(train_text, train_class)
    #print("Best score: %0.3f" % grid_search.best_score_)
    #print("Best parameters set:")
    #best_parameters = grid_search.best_estimator_.get_params()
    #for param_name in sorted(parameters.keys()):
        #print("\t%s: %r" % (param_name, best_parameters[param_name]))
print 'Avg accuracy: %.2f' % (sum(scores)/len(scores))



0.57852319756
0.579612284905
0.590720975822
0.571895424837
0.575163398693
0.581045751634
Avg accuracy: 0.58
